In [ ]:
from Bio.Seq import Seq
from Bio import SeqIO
import os

here = '/path/to/wkdir'
fa_dir = os.path.join(here, 'reference', 'mm10')
proj_dir = os.path.join(here, 'juliane_dac')

def parseFastq(path):
    seq = ""
    with open(path, 'r') as infile:
        next(infile)
        for line in infile:
            seq += line.strip()
    return seq

custom_ref=[]
with open(os.path.join(proj_dir, 'custom_genome_position.txt'), 'rt') as f:
    for line in f:
        var = line.split()
        custom_ref.append(var[0:])
        ins, chrom, pos = var[:3]
        customChrom = var[1] + '_' + var[0]
        
        ## make new ref dir
        outPath = os.path.join(proj_dir, '_ref', var[0])
        if not os.path.exists(outPath):
            os.mkdir(outPath)
            
        ## parse insert sequence as string
        insertSeq = parseFastq(os.path.join(proj_dir, var[0] + '.fastq'))
            
        ## parsing original fasta to write new file w/ desired insertion
        with open(os.path.join(outPath, customChrom + '.fa'), 'w') as out:
            for rec in SeqIO.parse(os.path.join(fa_dir, chrom + '.fa'), 'fasta'):
                rec.id = customChrom
                str_rec = str(rec.seq)
                rec.seq = Seq(str_rec[0:int(pos)] + insertSeq + str_rec[int(pos):])
                SeqIO.write(rec, out, 'fasta')